In [1]:
def run_get_auto_vars(cuil):
    """ funcion que a partir de un cuil y fecha corre score incremental"""
    import pyodbc 
    # Some other example server values are
    # server = 'localhost\sqlexpress' # for a named instance
    # server = 'myserver,port' # to specify an alternate port
    server = 'tcp:190.221.2.14' 
    database = 'felitest' 
    username = 'felipe' 
    password = 'f3l1p3' 
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()
    # Selected Query.
    query = """
    declare @cuil numeric(11) = {}; exec feliTest.dbo.sp_get_auto_variables @cuil """.format(cuil)
    cursor.execute(query)
    cnxn.commit()
    # Close and delete cursor
    cursor.close()
    del cursor
    # Close Connection
    cnxn.close() 
       
    
def run_get_auto_vars_df(df,cuil_column_name,processes=50):
    """Funcion que envia N store procedures al servidor para paralelizar el calculo.
    Modo de uso:
    import siisa as si
    si.run_score_df(df,'cuil','fecha')"""
    import multiprocessing
    from itertools import product
    # topear el numero de threads al shape del df (n casos) si es menor que processes.
    N = df.shape[0] 
    if N < processes:
        processes = N
    cuil_lst = df[[cuil_column_name]].to_records(index=False).tolist()
    with multiprocessing.Pool(processes=processes) as pool:
        results = pool.starmap(run_get_auto_vars, cuil_lst)
    return ('Done.')

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('correr.csv')

In [4]:
df.shape

(176681, 1)

In [5]:
df.head()

,cuil
0,20077683942
1,20077677411
2,20077660098
3,20202801456
4,20077647857


In [6]:
run_get_auto_vars_df(df,'cuil')

Process ForkPoolWorker-41:
Traceback (most recent call last):
  File "/home/elias/.conda/envs/siisa/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/elias/.conda/envs/siisa/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/elias/.conda/envs/siisa/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/elias/.conda/envs/siisa/lib/python3.7/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "<ipython-input-1-073f52f168ad>", line 16, in run_get_auto_vars
    cursor.execute(query)
KeyboardInterrupt


KeyboardInterrupt: 